# Инициализация

In [8]:
import pandas as pd
import pymssql
import cx_Oracle
from tqdm import tqdm
import re
import datetime
import numpy as np
from datetime import datetime, timedelta
from pandas import ExcelWriter
import itertools
from pandas.testing import assert_frame_equal
from functools import wraps


# plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import seaborn as sns
import matplotlib.pyplot as plt

import wordcloud
from wordcloud import WordCloud
from PIL import Image # converting images into arrays

pio.renderers.default = 'iframe+vscode'  # не знаю почему в jupyter без него не показывались графики

pd.set_option('display.max_columns', 60) # set so can see all columns of the DataFrame

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import accuracy_score, roc_auc_score,  recall_score, precision_recall_curve
from sklearn.preprocessing import StandardScaler

import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_selection import f_classif, chi2, f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.utils import class_weight as sk_class_weight

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cluster import AgglomerativeClustering

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
import scipy.stats.distributions as distributions
import scipy.stats as stats


import os, shutil,  os.path, tarfile
import networkx as nx

from pandas.core.dtypes.common import is_timedelta64_dtype

from abc import ABC, abstractmethod, abstractproperty

In [ ]:
#display_side_by_side вывод несколько таблиц
from IPython.display import display_html
from itertools import chain,cycle
def html_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display: inline-block"')
        html_str+='</td></th>'
    return html_str

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display: inline-block"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [ ]:
def prError(s): print("\033[91m {}\033[00m" .format(s))

def Check_length(len1, len2):
    
    # assert len1 == len2, 'Длина различна {} versus {}'.format(len1, len2)
    
    if (len1 != len2):
        prError('Длина различна {}  -  {}'.format(len1, len2))
        
def Check_dupl(n, duplicates):
    if (n != 0):
        prError('Ошибка. Количество дубликатов {}'.format(n))  
        display_side_by_side(duplicates.head())  

def identify_duplicates(df, columns):
    """identifies duplicates in columns of DataFrame """
 
    rows = df.groupby(columns).size().reset_index()
    rows.rename({0: 'row_nums'}, axis = 1, inplace = True)

    dupl = rows[rows['row_nums'] > 1] # дубликаты
    
    return len(dupl), pd.merge(df, dupl, how = 'inner', on = columns)

In [1]:
def combine_plotly_figs_to_html(plotly_figs, html_fname, include_plotlyjs='cdn', 
                                separator=None, auto_open=False):
    with open(html_fname, 'w', encoding="utf-8") as f:
        f.write(plotly_figs[0].to_html(include_plotlyjs=include_plotlyjs))
        for fig in plotly_figs[1:]:
            if separator:
                f.write(separator)
            if isinstance(fig, str):
                f.write(fig)
            else:
                f.write(fig.to_html(full_html=False, include_plotlyjs=False))

    if auto_open:
        import pathlib, webbrowser
        uri = pathlib.Path(html_fname).absolute().as_uri()
        webbrowser.open(uri)
# combine_plotly_figs_to_html([fig1, fig2], 'p_graph2.html', separator = 'Second option', auto_open=True)

In [ ]:
"""Очистить содержимое папки, используется для iframe_figures"""
def clear_folder(folder):
    if not os.path.isdir(folder):
        return
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

""" Пример
name = pre+"data/external/20230131_portfolio.pkl"
targz_name = pre+"data/external/20230131_portfolio.tar.gz"
make_tarfile(targz_name, name)
"""

def extract_all(archive, extract_path):
    if archive.endswith("tar.gz"):
        tar = tarfile.open(archive, "r:gz")
        tar.extractall(extract_path)
        tar.close()
    elif archive.endswith("tar"):
        tar = tarfile.open(archive, "r:")
        tar.extractall(extract_path)
        tar.close()

""" Пример 
name = pre + 'data/external/20230131_portfolio.tar.gz'
extract_all(name, pre +'data/interm/external_pkl')   
"""     

In [ ]:
def check_fields(df, etalon, column_to_check):
    df_filled = df[df[etalon].notnull()]
    mask = (df_filled[etalon]!=df_filled[column_to_check])

    tmp = df_filled[mask]

    if (len(tmp)>0):
        print(len(tmp))
    return tmp


In [ ]:

class ComparisonDataFrames():
    
    def __init__(self, data_1, data_2, **kwargs):
        
        actual = kwargs.get('actual', None)
        sourcesName = kwargs.get('sourcesName', ['df_1', 'df_2'])
        suffix = kwargs.get('suffix', '_df2')
        on = kwargs.get('on', None)
        left_on = kwargs.get('left_on', None)
        right_on = kwargs.get('right_on', None) 
        base_col = kwargs.get('base_col', None) 
        notnull_col = kwargs.get('notnull_col', None) 
        
        if on is not None:
            left_on = on
            right_on = on
        
        notes = pd.DataFrame(data = data_1.columns, columns=['Поле'])
        notes['Источник'] = sourcesName[0]
        if actual:
            notes['Дата актуальности'] = actual[0]

        tmp = pd.DataFrame(data = data_2.columns, columns=['Поле'])
        tmp['Источник'] = sourcesName[1]
        if actual:
            tmp['Дата актуальности'] = actual[1]
        notes = notes.append(tmp, ignore_index=True)
        
        self.data_1 = data_1
        self.data_2 = data_2
        self.notes = notes
        self.left_on = left_on
        self.right_on = right_on
        self.suffix = suffix
        self.sourcesName = sourcesName
        self.full_merge = None
        self.inner_merge = None
        self.base_col = base_col
        self.notnull_col = notnull_col
        
        error = self.check_duplicates()
        
        if not error:
            self.full_merge = self.merge()
            self.inner_merge = self.inner()

    def check_duplicates(self):
        
        error = False
        num, duplicates = identify_duplicates(self.data_1, self.left_on)
        if (num != 0):
            error = True
            prError('Ошибка. Количество дубликатов {} в {}'.format(num, self.sourcesName[0]))  
            display_side_by_side(duplicates.head())  
        
        num, duplicates = identify_duplicates(self.data_2,  self.right_on)
        if (num != 0):
            error = True
            prError('Ошибка. Количество дубликатов {} в {}'.format(num, self.sourcesName[1]))  
            display_side_by_side(duplicates.head())  

        return error
           
    def merge(self):
        
        full_merge = pd.merge(self.data_1, self.data_2, how = 'outer', 
                              left_on=self.left_on, right_on=self.right_on, 
                              suffixes=['', self.suffix])
        # определяем дубликаты по ID
        num, duplicates = identify_duplicates(full_merge, self.left_on)
        Check_dupl(num, duplicates)
        
        num, duplicates = identify_duplicates(full_merge, self.right_on)
        Check_dupl(num, duplicates)

        return full_merge
    
    def inner(self):
        
        inner_merge = pd.merge(self.data_1, self.data_2, how = 'inner', 
                              left_on=self.left_on, right_on=self.right_on, 
                              suffixes=['', self.suffix])
        return inner_merge
 
    def missedRows(self, num_df=0, sourcesName=None, **kwargs):
        add_condition = kwargs.get('add_condition', True)
        
        if sourcesName:
            num_df = self.sourcesName.index(sourcesName)
        key_col = self.notnull_col[num_df]
        
        # проверяем, что к notnull не было suffix
        if num_df==1 and (key_col + self.suffix in self.full_merge.columns):
            key_col = key_col + self.suffix 
            print('Проверка по столбцу {}'.format(key_col))

        # приложений, которых нет в dataframe
        mask_miss = (self.full_merge[key_col].isnull())

        missed = self.full_merge[mask_miss & (add_condition)]
        print("отсутствуют в {} {} строк".format(self.sourcesName[num_df], len(missed)))
        
        return missed
    
    def presentRows(self, **kwargs):
                
        # приложения, которые присутствуют в обоих dataframe
        print("одновременно присутствуют в {} и {} {} строк".format(self.sourcesName[0], self.sourcesName[1],  len(self.inner_merge)))
        
        return self.inner_merge
    
    def check_fields(self, column_1, column_2, **kwargs):
        
        add_condition = kwargs.get('add_condition', True)
        restriction = "add_condition" in kwargs
        
        df = self.inner_merge
        if column_2 + self.suffix in self.inner_merge.columns:
            column_2 = column_2 + self.suffix 
            print('Проверка по столбцам {} и {}'.format(column_1, column_2))
        
        mask = (df[column_1]!=df[column_2])

        notCoincide = df[mask & (add_condition)].copy()

        
        if (len(notCoincide)>0):
            print('значения не совпадают у {} строк из {}'.format(len(notCoincide), len(df)))
        else:
            print('значения совпадают во всех {} строках'.format(len(df)))
        
        if restriction:
           print('с учетом ограничения проверялось {} строк'.format(len(df[(add_condition)])))
           
        if self.base_col:
            keep_col = self.base_col.copy()
            keep_col.append(column_1)
            keep_col.append(column_2)  
              
        return notCoincide[keep_col]  if self.base_col else notCoincide
# kwargs = dict( actual = [portfolio['REPORT_DT'].min(), portfolio_tsal['REPORT_DT'].min()], 
#                sourcesName = ['portfolio', 'tsal_leas_portfolio'], 
#                suffix = '_tsal', on= ['CONTRACT_KEY']
#              )

# comp =  ComparisonDataFrames(portfolio, portfolio_tsal, **kwargs)    

In [ ]:
from string import Formatter
def strfdelta(tdelta, fmt='{D:02}d {H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-
    formatted string, just like the stftime() method does for datetime.datetime
    objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('Y', 'W', 'D', 'H', 'M', 'S')
    constants = {'Y': 31536000, 'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    return f.format(fmt, **values)

In [ ]:
def nvl(var, val):
  if var is None:
    return val
  return var

In [ ]:
def get_data(df, column, values):
    if isinstance(values, str):
        values = [values]
    return df[df[column].isin(values)]

def get_data_by_inn(df, inns):
    list_of_inn = ['new_lessee_inn', 'ИНН контрагента', 'NEW_INN', 'INN', 'ИНН']
    
    column_inn = list(set(list_of_inn) & set(df.columns))[0]
    return get_data(df, column_inn, inns)

def get_contract_by_deal(df, deals):
    return get_data(df, 'Номер сделки', deals)

def get_data_by_application(df, applications):
    return get_data(df, 'Приложение', applications)

def get_data_by_name(df, contract_names):
    list_of_contract_name = ['Номер договора (кратко)', 'Номер договора', 'CONTRACT_NUM']
    
    column_contract_name = list(set(list_of_contract_name) & set(df.columns))[0]
    return get_data(df, column_contract_name, contract_names)

In [ ]:
def read_from_oracle(sql_query, title='', params=dict()):
    
    cnxn_oracle = cx_Oracle.connect(obi_login, obi_pw, "10.0.2.45:1521/UAKRPDB", encoding="UTF-8") 
    
    try:
        df = pd.read_sql(sql_query, cnxn_oracle, params=params)
        print("Размер считанных данных {} - {}".format(title, df.shape))

    finally:
        cnxn_oracle.close() 
        
    return df